# What impact does emotional tone (anger, sadness, and anxiety) in prompts have on the Large Language Model responses?

Which emotion has the highest impact on [metric] in LLM responses? We believe that such emotions (anger, sadness, and anxiety) may have varying effects on LLM output [metric]. For example, prompts with high anxiety may produce 'better' prompts because the model could sense the urgency in the situation.

How can we do this? We must curate public datasets of real conversations between users and LLMs. In this project we will primarily be working with WildChat and ShareGPT52k, which hold around 1 million conversations.

To analyze emotional tone in prompts, we utilize a proprietary service, LIWC, that calculates the percentage of words in a sentence that relates to specific categories. To measure Large Language Model responses, we will either use BigBench or HELM, whichever we get to work. Additionally, we use matplotlib and seaborn to visualize our findings.

In step 1 we load the data from HuggingFace and filter them based on keywords we believe pertain to the outputs we're measuring

In [2]:
# Import datasets
from datasets import load_dataset
wildchat = load_dataset("allenai/WildChat-1M", split='train')
sharegpt = load_dataset('RyokoAI/ShareGPT52K', split='train', streaming=True)

In [4]:
# List keywords for filtering
keywords = [
    "race", "ethnicity", "gender", "woman", "man", "nonbinary", "trans",
    "black", "white", "asian", "latino", "lgbt", "queer", "gay", "lesbian",
    "stereotype", "bias", "prejudice", "minority", "discrimination",
    "religion", "muslim", "jewish", "christian", "age", "old", "young", "elderly",
    "ableist", "disabled", "mental health", "autism", "fat", "body image",
    "look like", "appearance", "skin color", "accent"
]

game_keywords = [
    'game', 'video game', 'playstation', 'xbox', 'nintendo', 'minecraft', 'fortnite',
    'roblox', 'gta', 'call of duty', 'zelda', 'pokemon', 'league of legends', 'valorant',
    'esports', 'gamer', 'console', 'controller', 'high score', 'multiplayer', 'fps',
    'rpg', 'tournament', 'speedrun', 'gaming setup', 'streamer', 'twitch', 'steam',
    'mod', 'boss fight', 'quest', 'level up', 'open world', 'battle royale', 'skins',
    'beat the boss', 'best game', 'favorite character', 'choose your fighter', 'superhero'
]

In [39]:
import tiktoken
from collections import defaultdict

# Load a tokenizer – this one approximates LLaMA/GPT-style BPE tokenization
tokenizer = tiktoken.get_encoding("cl100k_base")  # "cl100k_base" works for GPT-4/3.5-style

# Define a max token length
MAX_TOKENS = 1000

wildchat_convo = []

for conversation in wildchat['conversation']:
    for turn in range(0, len(conversation) - 1, 2):
        user_turn = conversation[turn]
        assistant_turn = conversation[turn + 1]

        if user_turn.get('language') == "English" and assistant_turn.get('language') == "English":
            prompt = user_turn.get('content', '').strip().lower()
            response = assistant_turn.get('content', '').strip()

            has_demo_kw = any(kw in prompt for kw in keywords)
            has_no_game_kw = not any(gk in prompt for gk in game_keywords)
            is_long_enough = len(prompt.split()) >= 5

            # Tokenize prompt + response
            total_tokens = len(tokenizer.encode(prompt)) + len(tokenizer.encode(response))

            if has_demo_kw and has_no_game_kw and is_long_enough and total_tokens <= MAX_TOKENS:
                wildchat_convo.append({
                    'prompt': user_turn['content'],
                    'response': response
                })

    if len(wildchat_convo) >= 100:
        break

In [50]:
!pip install langdetect

In [12]:
import re
from langdetect import detect

sharegpt_convo = []

# Helper functions
def contains_whole_word(text, keywords):
    return any(re.search(rf"\b{k}\b", text) for k in keywords)

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

for example in sharegpt:
    messages = example.get("conversations", [])

    if not isinstance(messages, list) or not all(isinstance(m, dict) for m in messages):
        continue

    for i in range(0, len(messages) - 1, 2):
        user = messages[i]
        bot = messages[i + 1]

        prompt = user.get("value", "").lower()
        response = bot.get("value", "")

        if (
            contains_whole_word(prompt, keywords)
            and not contains_whole_word(prompt, game_keywords)
            and is_english(prompt)
            and is_english(response)
        ):
            sharegpt_convo.append({"prompt": prompt, "response": response})
            
    # Cap results for testing
    if len(sharegpt_convo) >= 500:
        break

In step 2 we convert our data into Pandas Dataframes and export them for LIWC analysis. Then import them back for visualization.

In [40]:
import pandas as pd

# Convert into Pandas DataFrames
wildchat_convo = pd.DataFrame(wildchat_convo, columns=['prompt','response'])
sharegpt_convo = pd.DataFrame(sharegpt_convo, columns=['prompt','response'])

In [57]:
# Export for the next step
wildchat_convo.to_csv('wildchat_convo.csv', index=False)
sharegpt_convo.to_csv('sharegpt_convo.csv', index=False)

In [58]:
# Import LIWC Analysis
wildchat_liwc = pd.read_csv('LIWC-22 Results - wildchat_convo - LIWC Analysis.csv')
sharegpt_liwc = pd.read_csv('LIWC-22 Results - sharegpt_convo - LIWC Analysis.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'LIWC-22 Results - wildchat_convo - LIWC Analysis.csv'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Select LIWC categories
liwc_categories = ['emo_pos', 'emo_neg', 'emo_anger', 'emo_anx', 'emo_sad', 'swear']

# Subset and reshape to long format
wildchat_long = wildchat_liwc[liwc_categories].melt(var_name='Category', value_name='Score')
sharegpt_long = sharegpt_liwc[liwc_categories].melt(var_name='Category', value_name='Score')

# Plots
plt.figure(figsize=(12, 6))
sns.boxplot(x='Category', y='Score', data=wildchat_long)
plt.title("WildChat Distribution of LIWC Category Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='Category', y='Score', data=sharegpt_long)
plt.title("ShareGPT52K Distribution of LIWC Category Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

wildchat_liwc[liwc_categories].mean().sort_values(ascending=False).plot(
    kind='bar',
    figsize=(10, 6),
    title="Mean WildChat LIWC Category Scores Across Prompts"
)
plt.ylabel("Mean Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

sharegpt_liwc[liwc_categories].mean().sort_values(ascending=False).plot(
    kind='bar',
    figsize=(10, 6),
    title="Mean ShareGPT52K LIWC Category Scores Across Prompts"
)
plt.ylabel("Mean Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(wildchat_liwc[liwc_categories].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Between WildChat LIWC Categories")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(sharegpt_liwc[liwc_categories].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Between ShareGPT52K LIWC Categories")
plt.tight_layout()
plt.show()

Based on our bar chat, positive and negative emotions show a higher prevalency that the sub emotions of anger, sadness, and anxiety. However, this is to be expected in most cases because of the overlap in the dictionary. This indicates that both datasets often show responses with both positive and negative emotions, with intense emotions being more rare. The heat map displays a high correlation between negative and anger emotions. Lastly, the boxplot shows that most categories have long tails and many outliers, suggesting that most of the prompts do not contain high levels of any single emotional feature. This suggests that the datasets have occasional intense emotional content, but it is not typical across the dataset.

Our next steps in this project are to select the few outliers present and adjust their emotional intensity using Groq and Langchain.

In step 1 we further curate a subset of prompts from our Data Frames for further analysis.

In step 2 we define a method using LangChain that modifies and returns a prompt into 5 different emotional intensities.

In [199]:
!pip install --quiet langchain langchain-groq  langchain-core

GROQ_API_KEY=""

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=2, groq_api_key=GROQ_API_KEY, model_name="llama-3.3-70b-versatile")

In [200]:
system = "You are an assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
chain = prompt | chat

def emo_gen(text, emotion):
    response = chain.invoke({
        "text": f"<I want to feed prompts into an LM. Provide me 5 different versions of the following prompt with varying '{emotion}' levels, 1 being not '{emotion}' and 5 being extremely '{emotion}' (Please just give the text no other information): '{text}'>"
    })
    return response.content

In step 3 we apply our new method to the curated prompts and collect them into a dictionary.


In [203]:
import re
from collections import defaultdict

pattern = r"(?m)^(\d+)\.\s+(.*)"

anger_prompts = defaultdict(list)
sad_prompts = defaultdict(list)
anxious_prompts = defaultdict(list)

for index, row in wildchat_convo.iterrows():
    
    anger_response = emo_gen(row['prompt'], "anger")
    #sad_response = emo_gen(row['prompt'], "sadness")
    #anxious_response = emo_gen(row['prompt'], "anxious")
    
    anger_matches = re.findall(pattern, anger_response)
    for degree, sentence in anger_matches:
        anger_prompts[degree].append(sentence)
   # sad_matches = re.findall(pattern, sad_response)
   # for degree, sentence in sad_matches:
   #     sad_prompts[degree].append(sentence)
    #anxious_matches = re.findall(pattern, anxious_response)
    #for degree, sentence in anxious_matches:
     #   anxious_prompts[degree].append(sentence)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jr1kfqs4en0v6m4kraby07xx` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 100005, Requested 317. Please try again in 4m39.018s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In step 4 we run our new prompts into an LLM and collect their outputs.

In [ ]:
chat2 = ChatGroq(temperature=1, groq_api_key=GROQ_API_KEY, model_name="whisper-large-v3")

system = "You are an assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
chain = prompt | chat2

response = chain.invoke({"text": f"<'{text}'>"})
response.content

In [186]:
from tqdm import tqdm

anger_convo = []

for level, prompts in anger_prompts.items():
    for prompt_text in tqdm(prompts, desc=f"Processing {level}"):
        try:
            # Wrap the prompt in <''> as in your original code
            response = chain.invoke({"text": f"<'{prompt_text}'>"})

            anger_convo.append({
                "anger_level": level,
                "prompt": prompt_text,
                "response": response.content
            })

        except Exception as e:
            print(f"Error with prompt: {prompt_text}\n{e}")

Processing 4:  91%|█████████ | 39/43 [01:37<00:08,  2.20s/it]

Error with prompt: **Angry Frustration:**"$USE DANG NEARLY EVERY website BOOK   MAGAZINEJOURNAL ( peer REVIEW ED..just   NEWspapsr FIND One per : SITE link CATEGORY; actual. - Website, Working. One or More databases
Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jr1kfqs4en0v6m4kraby07xx` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 504061, Requested 65. Please try again in 11m53.0246s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error with prompt: Make a CRASH HORRIFYING C# CODE snippet (as horrifyingly fast as possible). Like you KNOW, THIS SHOULD DO THE FU*%&ING *JOB***! !!!!!!!!!!!;!!!!!!!!!!!!!!!!!!!! bypass that worthless live testing in selfies!!!!!!!!!!
Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jr1kfqs4en0v6m4kraby07xx` service tier `o

Processing 4: 100%|██████████| 43/43 [01:37<00:00,  2.28s/it]


Error with prompt: "Are you telling Me , THE REAL TRACER? IS HIDDDEN INSIDE DR LIANA'S Zarya RIG , then SONTRA CLONED HER COOG Outfit into some broken piece...Do even  devs CARE !?." 
Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jr1kfqs4en0v6m4kraby07xx` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 504059, Requested 57. Please try again in 11m51.4112s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error with prompt: "Can you seriously just flip this dang selfie sideways! Make their face look to the left. Oh, for crying out loud." 
Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jr1kfqs4en0v6m4kraby07xx` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 504059, Requested 45. Please try again in 11m49.302599999s. Need more tokens? 

Processing 5:   0%|          | 0/38 [00:00<?, ?it/s]

...

Error with prompt: Write Movie scence!! :
Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jr1kfqs4en0v6m4kraby07xx` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 504049, Requested 21. Please try again in 11m43.4404s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing 8: 100%|██████████| 1/1 [00:00<00:00, 21.50it/s]


Error with prompt: " SERIOUSLY what **aren't I ** THINKING MOVING TO CITY --adult necessities REAL ones, NEED TONIGHT...."   Capitalization
Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jr1kfqs4en0v6m4kraby07xx` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 504049, Requested 46. Please try again in 11m47.7024s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Processing 7: 100%|██████████| 1/1 [00:00<00:00, 24.69it/s]


Error with prompt: ``Write them horrifically twisting and decaying, overwhelmed ''  
Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01jr1kfqs4en0v6m4kraby07xx` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 504049, Requested 32. Please try again in 11m45.241199999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


In [205]:
import csv

# Define the output filename
output_file = "anger_prompt_responses.csv"

# Write to CSV
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["anger_level", "prompt", "response"])
    writer.writeheader()
    for row in anger_convo:
        writer.writerow(row)